# ResNet

This software uses cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline

See MIT License in https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline README.md

Wang, Z., Yan, W. and Oates, T. (2017) ‘Time series classification from scratch with deep neural networks: A strong baseline’, 2017 International Joint Conference on Neural Networks (IJCNN), pp. 1578–1585 Online.


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-


import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ReduceLROnPlateau


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, RepeatedKFold

import os
import pathlib
import time
from datetime import datetime

np.random.seed(813306)

# User inputs ##

flist = ['GunPoint'] # List dataset directory names.
batch_size = 64 # Wang: int(min(x_train.shape[0]/10, 16)) 
nb_epochs = 1500 # Wang: 1500
k = 2 # For k-fold cross validation. If k=1, the original test-train split is used.
m = 1 # Number of repetitions of k-fold cross validation (if k>1).
k_fold_seed = 87
tensorboard = True # Set to True to write logs for use by TensorBoard
# Directories
fdir = '../data'  
logs_dir = '../logs'
tensorboard_dir = '../logs/tensorboard'
timestamp = '{:%Y-%m-%dT%H:%M}'.format(datetime.now())
logs_dir = logs_dir +'/' + timestamp
tensorboard_dir = tensorboard_dir +'/' + timestamp

def build_resnet(input_shape, n_feature_maps, nb_classes):
    print ('build conv_x')
    x = Input(shape=(input_shape))
    conv_x = keras.layers.BatchNormalization()(x)
    conv_x = keras.layers.Conv2D(n_feature_maps, 8, 1, padding='same')(conv_x)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps, 1, 1,padding='same')(x)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x)
    print ('Merging skip connection')
    y = keras.layers.add([shortcut_y, conv_z])
    y = Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.add([shortcut_y, conv_z])
    y = Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.add([shortcut_y, conv_z])
    y = Activation('relu')(y)
     
    full = keras.layers.GlobalAveragePooling2D()(y)   
    out = Dense(nb_classes, activation='softmax')(full)
    print ('        -- model was built.')
    return x, out
 
       
def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
   


    
    
def train_model(fname, x_train, y_train, x_test, y_test, label="0"):
    print('Running dataset', fname)
    nb_classes = len(np.unique(y_test))
     
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
     
     
    Y_train = utils.to_categorical(y_train, nb_classes)
    Y_test = utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
      
    x_test = (x_test - x_train_mean)/(x_train_std)
    x_train = x_train.reshape(x_train.shape + (1,1,))
    x_test = x_test.reshape(x_test.shape + (1,1,))
     
     
    x , y = build_resnet(x_train.shape[1:], 64, nb_classes)
    model = Model(x, y)
    #print(model.summary())
    
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
      
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
    callbacks = [reduce_lr]
    if tensorboard:
        tb_dir = tensorboard_dir+'/'+fname+'_'+label
        pathlib.Path(tb_dir).mkdir(parents=True, exist_ok=True) 
        print('Tensorboard logs in', tb_dir)
        callbacks = [reduce_lr, keras.callbacks.TensorBoard(log_dir=tb_dir, histogram_freq=0)]

    start = time.time()
    hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks=callbacks)
    end = time.time()
    log = pd.DataFrame(hist.history)
    
    # Print and save results. Print the testing results which has the lowest training loss.
    print('Training complete on', fname)
    duration_minutes = str(round((end-start)/60))
    print('Training time ', end-start, 'seconds, which is about', duration_minutes, 'minutes.')    
    print('Selected the test result with the lowest training loss. Loss and validation accuracy are -')
    idx = log['loss'].idxmin()
    loss = log.loc[log['loss'].idxmin]['loss']
    val_acc = log.loc[log['loss'].idxmin]['val_acc']
    print(loss, val_acc, 'at index', str(idx), ' (epoch ', str(idx+1), ')')
    summary = '|' + label + '  |'+str(loss)+'  |'+str(val_acc)+' |'+str(idx)+' |'+ duration_minutes + 'mins  |'
    summary_csv = label+','+str(loss)+','+str(val_acc)+','+str(idx)+','+ duration_minutes 
    
    # Save summary file and log file.
    print('Tensorboard logs in', tb_dir)
    pathlib.Path(logs_dir+'/'+fname).mkdir(parents=True, exist_ok=True) 
    with open(logs_dir+'/'+fname+'/resnet_summary.csv', 'a+') as f:
        f.write(summary_csv)
        f.write('\n')
        print('Added summary row to ', logs_dir+'/'+fname+'/resnet_summary.csv')  
    print('Saving logs to',logs_dir+'/'+fname+'/history_'+label+'.csv')
    log.to_csv(logs_dir+'/'+fname+'/history_'+label+'.csv')
    
    return summary, model


# main
results = []
for each in flist:
    fname = each
    x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
    x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
    # k-fold cross validation setup
    if k > 1:
        x_all = np.concatenate((x_train, x_test), axis=0)
        y_all = np.concatenate((y_train, y_test), axis=0)
        kfold = RepeatedKFold(n_splits=k, n_repeats=m, random_state=k_fold_seed)
        count = 0
        for train, test in kfold.split(x_all):
            x_train, y_train, x_test, y_test = x_all[train], y_all[train], x_all[test], y_all[test]
            summary, model = train_model(fname, x_train, y_train, x_test, y_test, str(count))
            results.append(summary)
            count = count + 1
    else:
        summary, model = train_model(fname, x_train, y_train, x_test, y_test)
        results.append(summary)
        
print('DONE')
print(fname, timestamp)
for each in results:
    print(each)
#print('Example prediction probabilities')
#y_predict = model.predict_proba(x_test)            
#for yp in y_predict[1:20]:
#    print(yp)

Running dataset GunPoint
build conv_x
build conv_y
build conv_z
Merging skip connection
build conv_x
build conv_y
build conv_z
Merging skip connection
build conv_x
build conv_y
build conv_z
Merging skip connection
        -- model was built.
Tensorboard logs in ../logs/tensorboard/2018-12-22T14:40/GunPoint_0
Train on 100 samples, validate on 100 samples
Epoch 1/1500
100/100 [==============================] - 2s 19ms/step - loss: 0.6914 - acc: 0.5300 - val_loss: 0.6974 - val_acc: 0.4800
Epoch 2/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.6624 - acc: 0.6800 - val_loss: 0.6969 - val_acc: 0.4800
Epoch 3/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.5603 - acc: 0.6500 - val_loss: 0.6953 - val_acc: 0.4800
Epoch 4/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.3234 - acc: 0.8200 - val_loss: 0.6935 - val_acc: 0.4700
Epoch 5/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.2902 - acc: 0.9100 - val_loss

100/100 [==============================] - 0s 1ms/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.7361 - val_acc: 0.5200
Epoch 60/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.7425 - val_acc: 0.5200
Epoch 61/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.7495 - val_acc: 0.5200
Epoch 62/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.7569 - val_acc: 0.5200
Epoch 63/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.7627 - val_acc: 0.5200
Epoch 64/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.7679 - val_acc: 0.5200
Epoch 65/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.7720 - val_acc: 0.5200
Epoch 66/1500
100/100 [==============================] - 0s 1ms/step 

100/100 [==============================] - 0s 1ms/step - loss: 3.5692e-04 - acc: 1.0000 - val_loss: 1.7568 - val_acc: 0.5200
Epoch 119/1500
100/100 [==============================] - 0s 1ms/step - loss: 5.2144e-04 - acc: 1.0000 - val_loss: 1.8204 - val_acc: 0.5200
Epoch 120/1500
100/100 [==============================] - 0s 1ms/step - loss: 6.3180e-04 - acc: 1.0000 - val_loss: 1.8841 - val_acc: 0.5200
Epoch 121/1500
100/100 [==============================] - 0s 1ms/step - loss: 3.9950e-04 - acc: 1.0000 - val_loss: 1.9503 - val_acc: 0.5200
Epoch 122/1500
100/100 [==============================] - 0s 1ms/step - loss: 4.1069e-04 - acc: 1.0000 - val_loss: 2.0209 - val_acc: 0.5200
Epoch 123/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.2472e-04 - acc: 1.0000 - val_loss: 2.0909 - val_acc: 0.5200
Epoch 124/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.9251e-04 - acc: 1.0000 - val_loss: 2.1582 - val_acc: 0.5200
Epoch 125/1500
100/100 [===========

100/100 [==============================] - 0s 1ms/step - loss: 3.7792e-04 - acc: 1.0000 - val_loss: 4.6978 - val_acc: 0.5200
Epoch 177/1500
100/100 [==============================] - 0s 1ms/step - loss: 3.3517e-04 - acc: 1.0000 - val_loss: 4.6546 - val_acc: 0.5200
Epoch 178/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.9670e-04 - acc: 1.0000 - val_loss: 4.6189 - val_acc: 0.5200
Epoch 179/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.1403e-04 - acc: 1.0000 - val_loss: 4.6051 - val_acc: 0.5200
Epoch 180/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.7896e-04 - acc: 1.0000 - val_loss: 4.5935 - val_acc: 0.5200
Epoch 181/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.1660e-04 - acc: 1.0000 - val_loss: 4.5824 - val_acc: 0.5200
Epoch 182/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.9102e-04 - acc: 1.0000 - val_loss: 4.5906 - val_acc: 0.5200
Epoch 183/1500
100/100 [===========

100/100 [==============================] - 0s 1ms/step - loss: 9.1665e-05 - acc: 1.0000 - val_loss: 3.8875 - val_acc: 0.5500
Epoch 235/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 3.7221 - val_acc: 0.5500
Epoch 236/1500
100/100 [==============================] - 0s 1ms/step - loss: 8.7407e-05 - acc: 1.0000 - val_loss: 3.6042 - val_acc: 0.5400
Epoch 237/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.2076e-04 - acc: 1.0000 - val_loss: 3.4979 - val_acc: 0.5400
Epoch 238/1500
100/100 [==============================] - 0s 1ms/step - loss: 4.8795e-04 - acc: 1.0000 - val_loss: 3.4306 - val_acc: 0.5400
Epoch 239/1500
100/100 [==============================] - 0s 1ms/step - loss: 5.5068e-04 - acc: 1.0000 - val_loss: 3.3695 - val_acc: 0.5400
Epoch 240/1500
100/100 [==============================] - 0s 1ms/step - loss: 8.8925e-05 - acc: 1.0000 - val_loss: 3.3266 - val_acc: 0.5500
Epoch 241/1500
100/100 [===============

...

100/100 [==============================] - 0s 1ms/step - loss: 4.2510e-04 - acc: 1.0000 - val_loss: 0.0425 - val_acc: 0.9700
Epoch 1245/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.0817e-04 - acc: 1.0000 - val_loss: 0.0405 - val_acc: 0.9700
Epoch 1246/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.7810e-04 - acc: 1.0000 - val_loss: 0.0396 - val_acc: 0.9700
Epoch 1247/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.5061e-04 - acc: 1.0000 - val_loss: 0.0390 - val_acc: 0.9700
Epoch 1248/1500
100/100 [==============================] - 0s 1ms/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0401 - val_acc: 0.9700
Epoch 1249/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.6744e-04 - acc: 1.0000 - val_loss: 0.0434 - val_acc: 0.9700
Epoch 1250/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.4243e-04 - acc: 1.0000 - val_loss: 0.0461 - val_acc: 0.9700
Epoch 1251/1500
100/100 [========

100/100 [==============================] - 0s 1ms/step - loss: 2.8045e-04 - acc: 1.0000 - val_loss: 0.0587 - val_acc: 0.9800
Epoch 1303/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.4559e-04 - acc: 1.0000 - val_loss: 0.0592 - val_acc: 0.9800
Epoch 1304/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.6020e-04 - acc: 1.0000 - val_loss: 0.0591 - val_acc: 0.9800
Epoch 1305/1500
100/100 [==============================] - 0s 1ms/step - loss: 3.2540e-04 - acc: 1.0000 - val_loss: 0.0586 - val_acc: 0.9800
Epoch 1306/1500
100/100 [==============================] - 0s 1ms/step - loss: 7.2140e-04 - acc: 1.0000 - val_loss: 0.0578 - val_acc: 0.9800
Epoch 1307/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.4104e-04 - acc: 1.0000 - val_loss: 0.0575 - val_acc: 0.9800
Epoch 1308/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.2269e-04 - acc: 1.0000 - val_loss: 0.0571 - val_acc: 0.9800
Epoch 1309/1500
100/100 [====

100/100 [==============================] - 0s 1ms/step - loss: 3.2162e-04 - acc: 1.0000 - val_loss: 0.0451 - val_acc: 0.9700
Epoch 1361/1500
100/100 [==============================] - 0s 1ms/step - loss: 4.9239e-04 - acc: 1.0000 - val_loss: 0.0451 - val_acc: 0.9700
Epoch 1362/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.2799e-04 - acc: 1.0000 - val_loss: 0.0447 - val_acc: 0.9700
Epoch 1363/1500
100/100 [==============================] - 0s 1ms/step - loss: 4.4798e-04 - acc: 1.0000 - val_loss: 0.0444 - val_acc: 0.9700
Epoch 1364/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.8874e-04 - acc: 1.0000 - val_loss: 0.0443 - val_acc: 0.9700
Epoch 1365/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.0650e-04 - acc: 1.0000 - val_loss: 0.0440 - val_acc: 0.9700
Epoch 1366/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.8000e-04 - acc: 1.0000 - val_loss: 0.0439 - val_acc: 0.9700
Epoch 1367/1500
100/100 [====

100/100 [==============================] - 0s 1ms/step - loss: 2.1952e-04 - acc: 1.0000 - val_loss: 0.0405 - val_acc: 0.9800
Epoch 1419/1500
100/100 [==============================] - 0s 1ms/step - loss: 3.3546e-04 - acc: 1.0000 - val_loss: 0.0416 - val_acc: 0.9800
Epoch 1420/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.1178e-04 - acc: 1.0000 - val_loss: 0.0423 - val_acc: 0.9700
Epoch 1421/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.6569e-04 - acc: 1.0000 - val_loss: 0.0429 - val_acc: 0.9700
Epoch 1422/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.6365e-04 - acc: 1.0000 - val_loss: 0.0434 - val_acc: 0.9700
Epoch 1423/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.1753e-04 - acc: 1.0000 - val_loss: 0.0440 - val_acc: 0.9700
Epoch 1424/1500
100/100 [==============================] - 0s 1ms/step - loss: 3.4886e-04 - acc: 1.0000 - val_loss: 0.0441 - val_acc: 0.9700
Epoch 1425/1500
100/100 [====

100/100 [==============================] - 0s 1ms/step - loss: 1.4288e-04 - acc: 1.0000 - val_loss: 0.0381 - val_acc: 0.9700
Epoch 1477/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.7162e-04 - acc: 1.0000 - val_loss: 0.0384 - val_acc: 0.9700
Epoch 1478/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.6788e-04 - acc: 1.0000 - val_loss: 0.0385 - val_acc: 0.9700
Epoch 1479/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.8475e-04 - acc: 1.0000 - val_loss: 0.0387 - val_acc: 0.9700
Epoch 1480/1500
100/100 [==============================] - 0s 1ms/step - loss: 2.5578e-04 - acc: 1.0000 - val_loss: 0.0390 - val_acc: 0.9700
Epoch 1481/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.5826e-04 - acc: 1.0000 - val_loss: 0.0395 - val_acc: 0.9700
Epoch 1482/1500
100/100 [==============================] - 0s 1ms/step - loss: 1.4823e-04 - acc: 1.0000 - val_loss: 0.0399 - val_acc: 0.9700
Epoch 1483/1500
100/100 [====